<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/Flood-Mapping-Sentinel1/blob/main/Flood_Mapping_Sentinel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import requests
from PIL import Image
from io import BytesIO
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker

In [2]:
ee.Authenticate()
ee.Initialize(project="ee-saeiddalirisu", opt_url='https://earthengine-highvolume.googleapis.com')

In [7]:
# Select the area of intrest
map=geemap.Map()
map.add_basemap('SATELLITE')  # Add satellite imagery as the basemap
map

In [5]:
roi = map.draw_last_feature.geometry()
#roi = roi.buffer(5000)  # Buffer ROI by 5 km
# Define event date
event_date_str = '2019-03-19'

In [7]:

event_date = datetime.strptime(event_date_str, '%Y-%m-%d')
# Speckle filter function
def speckle(img):
    return img.focal_median(100, 'square', 'meters') \
              .copyProperties(img, img.propertyNames())

# Load and filter Sentinel-1 collection (descending pass only)
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filterBounds(roi)
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
              .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')))


# Get available dates before and after event
dates_before = (collection
                .filterDate('2017-01-01', event_date_str)
                .aggregate_array('system:time_start')
                .getInfo())

dates_after = (collection
               .filterDate((event_date + timedelta(days=1)).strftime('%Y-%m-%d'), '2024-12-31')
               .aggregate_array('system:time_start')
               .getInfo())

# Helper to get unique dates in YYYY-MM-DD
def unique_dates(timestamps):
    return sorted(list(set([datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in timestamps])))

before_days = unique_dates(dates_before)
after_days = unique_dates(dates_after)

# Get closest valid days
last_day_before = before_days[-1]
first_day_after = after_days[0]

print('Event day:', event_date_str)
print('Last day before event:', last_day_before)
print('First day after event:', first_day_after)

# Filter images on those specific days
before_images = collection.filterDate(last_day_before, (datetime.strptime(last_day_before, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'))
after_images = collection.filterDate(first_day_after, (datetime.strptime(first_day_after, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'))

# Count the number of images before and after the event
num_before = before_images.size().getInfo()
num_after = after_images.size().getInfo()

print(f'Number of Sentinel-1 images BEFORE the event on {last_day_before}: {num_before}')
print(f'Number of Sentinel-1 images AFTER the event on {first_day_after}: {num_after}')

# Apply speckle filter
before_filtered = before_images.map(speckle)
after_filtered = after_images.map(speckle)

# Mosaic the filtered images
before_mosaic = before_filtered.mosaic().clip(roi)
after_mosaic = after_filtered.mosaic().clip(roi)

# Optional: Create difference image
difference = after_mosaic.subtract(before_mosaic)

# Visualize Sentinel-1 scene footprints
def get_footprints(img):
    return ee.Feature(img.geometry())

before_footprints = before_images.map(get_footprints)
after_footprints = after_images.map(get_footprints)

# -------------------------------------------------------------------------------

# Otsu thresholding function
def otsu(histogram):
    total = 0
    for b in range(len(histogram['histogram'])):
        total += histogram['histogram'][b]
    sum_total = 0
    for b in range(len(histogram['histogram'])):
        sum_total += histogram['histogram'][b] * histogram['bucketMeans'][b]

    sumB = 0
    wB = 0
    varMax = 0
    threshold = 0

    for i in range(len(histogram['histogram'])):
        wB += histogram['histogram'][i]
        if wB == 0:
            continue
        wF = total - wB
        if wF == 0:
            break
        sumB += histogram['histogram'][i] * histogram['bucketMeans'][i]
        mB = sumB / wB
        mF = (sum_total - sumB) / wF
        varBetween = wB * wF * (mB - mF) ** 2
        if varBetween > varMax:
            varMax = varBetween
            threshold = histogram['bucketMeans'][i]

    return threshold

# Compute histogram over the ROI
histogram = difference.select('VV').reduceRegion(
    reducer=ee.Reducer.histogram(255),
    geometry=roi,
    scale=30,
    bestEffort=True,
    maxPixels=1e9
).get('VV').getInfo()

# Apply Otsu
threshold = otsu(histogram)
print(f"Otsu Threshold: {threshold:.2f}")

# Extract flooded pixels
flooded = difference.select('VV').lt(threshold).selfMask()

# Initialize the map (optional, for interactive use)
domain = ee.Image().byte().paint(featureCollection=roi, color=0, width=1)
boundary_vis = domain.visualize(palette=['black'])
bounds = roi.bounds().coordinates().get(0)
coords = bounds.getInfo()
lons = [pt[0] for pt in coords]
lats = [pt[1] for pt in coords]
lon_min, lon_max = min(lons), max(lons)
lat_min, lat_max = min(lats), max(lats)
Map = geemap.Map(center=[(lat_min + lat_max) / 2, (lon_min + lon_max) / 2], zoom=10)
Map.add_basemap('SATELLITE')

change = before_mosaic.subtract(after_mosaic).rename('flood')

vis_vv = {'min': -20, 'max': 0, 'palette': ['black', 'white']}
vis_flood = {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}

Map.addLayer(before_footprints.style(**{'color': 'cyan', 'fillColor': '00000000'}), {}, 'S1 Footprints Before', False)
Map.addLayer(after_footprints.style(**{'color': 'yellow', 'fillColor': '00000000'}), {}, 'S1 Footprints After', False)
Map.addLayer(before_mosaic.select('VV'), {'min': -20, 'max': 0}, f'Speckled Mosaic Before {last_day_before}', False)
Map.addLayer(after_mosaic.select('VV'), {'min': -20, 'max': 0}, f'Speckled Mosaic After {first_day_after}', False)
Map.addLayer(difference.select('VV'), {'min': -20, 'max': 0}, f'Speckled Changed', False)
Map.addLayer(flooded, {'palette': ['red']}, 'Flooded Area (Otsu)', False)
Map


In [7]:
# ------------------- Sentinel-2 basemap -------------------

sentinel2_img = (ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(roi)
    .filterDate('2020-01-01', '2021-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
    .median()
    .clip(roi)
    .visualize(bands=['B4', 'B3', 'B2'], min=500, max=3000))

# Visualization params
vis_vv = {'min': -20, 'max': 0, 'palette': ['black', 'white']}
vis_diff = {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}
vis_flood = {'palette': ['blue']}

def get_thumb_url(image, vis_params, region, dimensions=512):
    url = image.getThumbURL({
        'region': region,
        'dimensions': dimensions,
        'min': vis_params.get('min'),
        'max': vis_params.get('max'),
        'palette': vis_params.get('palette'),
        'format': 'png'
    })
    return url

def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Get thumbnail URLs
before_url = get_thumb_url(before_mosaic.select('VV'), vis_vv, roi)
after_url = get_thumb_url(after_mosaic.select('VV'), vis_vv, roi)
diff_url = get_thumb_url(difference.select('VV'), vis_diff, roi)
flood_url = get_thumb_url(flooded, vis_flood, roi)
sentinel2_url = get_thumb_url(sentinel2_img, {'min': 10, 'max': 255}, roi)

# Download images
before_img = download_image(before_url)
after_img = download_image(after_url)
diff_img = download_image(diff_url)
flood_img = download_image(flood_url)
sentinel2_img_pil = download_image(sentinel2_url)

# Prepare list for looping in plots
images = [
    (before_img, f' Before Event Image ({last_day_before})'),
    (after_img, f'After Event Image ({first_day_after})'),
    (diff_img, 'Difference Image (After - Before)'),
    # Flood subplot handled separately
]

# --------- Plotting with coordinates ----------

extent = [lon_min, lon_max, lat_min, lat_max]

fig, axs = plt.subplots(2, 2, figsize=(12, 8))

for i, ax in enumerate(axs.flat):
    if i == 3:
        # Sentinel-2 basemap + flood overlay
        ax.imshow(sentinel2_img_pil, extent=extent)
        ax.imshow(flood_img, alpha=0.5, extent=extent)
        ax.set_title('Flooded Area')
    else:
        # Before, After, Difference images
        ax.imshow(images[i][0], extent=extent)
        ax.set_title(images[i][1])

    # Set labels
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Set axis limits
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    # Set ticks with 5 intervals
    ax.set_xticks(np.linspace(lon_min, lon_max, 5))
    ax.set_yticks(np.linspace(lat_min, lat_max, 5))

    # Show grid with light dashed lines
    ax.grid(True, linestyle='--', alpha=0.5)

    # Format tick labels to 2 decimal places
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))

plt.tight_layout()
plt.show()
